<a href="https://colab.research.google.com/github/inachi19951023/mysite/blob/master/saunaikitai_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import time
import re

In [ ]:
"""
GCSへの書き込みをするための
"""

In [ ]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数
        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト
        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """
        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/race/" + race_id
                # スクレイピング
                html = requests.get(url)
                html.encoding = "EUC-JP"
                # メインとなるテーブルデータを取得
                df = pd.read_html(html.text)[0]
                # 列名に半角スペースがあれば除去する
                df = df.rename(columns=lambda x: x.replace(' ', ''))
                # 天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                soup = BeautifulSoup(html.text, "html.parser")
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text] * len(df)
                    if "障" in text:
                        df["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text] * len(df)
                    if "年" in text:
                        df["date"] = [text] * len(df)
                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list
                #インデックスをrace_idにする
                df.index = [race_id] * len(df)
                race_results[race_id] = df
            #存在しないrace_idを飛ばす
            except IndexError:
                continue
            except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #Jupyterで停止ボタンを押した時の対処
            except:
                break
        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])
        return race_results_df




In [ ]:
race_id_list=[]
#取得する年数
year = "2023"
#競馬場
places=["05","06","08","09"]
counts=["01","02","03","04","05","06"]
days=["01","02","03","04","05","06","07","08","09","10","11","12"]
#races=["11"]
races=["01","02","03","04","05","06","07","08","09","10","11"]
#開催回数分ループ
race_id = ''
for place in range(len(places)):
  for count in range(len(counts)):
    for day in range(len(days)):
      for race in range(len(races)):
        race_id = ''
        race_id = year+places[place]+counts[count]+days[day]+races[race]
        race_id_list.append(race_id)
print(race_id_list)

['202305010101', '202305010102', '202305010103', '202305010104', '202305010105', '202305010106', '202305010107', '202305010108', '202305010109', '202305010110', '202305010111', '202305010201', '202305010202', '202305010203', '202305010204', '202305010205', '202305010206', '202305010207', '202305010208', '202305010209', '202305010210', '202305010211', '202305010301', '202305010302', '202305010303', '202305010304', '202305010305', '202305010306', '202305010307', '202305010308', '202305010309', '202305010310', '202305010311', '202305010401', '202305010402', '202305010403', '202305010404', '202305010405', '202305010406', '202305010407', '202305010408', '202305010409', '202305010410', '202305010411', '202305010501', '202305010502', '202305010503', '202305010504', '202305010505', '202305010506', '202305010507', '202305010508', '202305010509', '202305010510', '202305010511', '202305010601', '202305010602', '202305010603', '202305010604', '202305010605', '202305010606', '202305010607', '202305

In [ ]:
df = Results.scrape(race_id_list)
df.to_csv("race_test.csv")

 34%|███▍      | 1080/3168 [29:58<1:02:42,  1.80s/it]